# 🔍 PowerApps Data Exploration

## Analyzing Sales Opportunities, Customer Feedback, and Inventory Data

This notebook explores data exported from PowerApps and processed through our ETL pipeline. We'll analyze patterns, generate insights, and create visualizations that would be valuable for business stakeholders.

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '${:,.2f}'.format(x) if x > 100 else '{:.2f}'.format(x))

print("✅ Libraries loaded successfully")

## 📊 Connect to Database

First, let's connect to the SQLite database created by our pipeline.

In [ ]:
# Connect to database
conn = sqlite3.connect('../data_warehouse.db')

# Load tables
opportunities = pd.read_sql_query("SELECT * FROM opportunities", conn)
feedback = pd.read_sql_query("SELECT * FROM customer_feedback", conn)
inventory = pd.read_sql_query("SELECT * FROM inventory", conn)

print(f"📊 Opportunities: {len(opportunities):,} records")
print(f"📊 Customer Feedback: {len(feedback):,} records")
print(f"📊 Inventory: {len(inventory):,} records")

# Close connection
conn.close()

## 🎯 Part 1: Sales Opportunities Analysis

Let's analyze the sales pipeline to understand deal flow, win rates, and revenue patterns.

In [ ]:
# Convert dates
opportunities['created_date'] = pd.to_datetime(opportunities['created_date'])
opportunities['close_date'] = pd.to_datetime(opportunities['close_date'])

# Display first few rows
print("📋 Sample Opportunities:")
opportunities.head()

In [ ]:
# Pipeline stage distribution
stage_counts = opportunities['stage'].value_counts()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Pie chart
axes[0].pie(stage_counts.values, labels=stage_counts.index, autopct='%1.1f%%', startangle=90)
axes[0].set_title('Opportunities by Pipeline Stage', fontsize=14, fontweight='bold')

# Bar chart with amounts
stage_amounts = opportunities.groupby('stage')['amount'].sum().sort_values(ascending=False)
axes[1].barh(stage_amounts.index, stage_amounts.values)
axes[1].set_xlabel('Total Amount ($)')
axes[1].set_title('Pipeline Value by Stage', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Print key metrics
total_pipeline = opportunities['amount'].sum()
weighted_pipeline = opportunities['weighted_amount'].sum()
print(f"💰 Total Pipeline Value: ${total_pipeline:,.2f}")
print(f"⚖️  Weighted Pipeline: ${weighted_pipeline:,.2f}")

In [ ]:
# Win/Loss Analysis
closed_deals = opportunities[opportunities['stage'].isin(['Closed Won', 'Closed Lost'])]

win_rate = len(closed_deals[closed_deals['stage'] == 'Closed Won']) / len(closed_deals) * 100 if len(closed_deals) > 0 else 0

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Win rate by region
region_performance = closed_deals.groupby('region').apply(
    lambda x: (x['stage'] == 'Closed Won').sum() / len(x) * 100
).sort_values(ascending=False)

axes[0].bar(region_performance.index, region_performance.values)
axes[0].set_title('Win Rate by Region', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Win Rate (%)')
axes[0].tick_params(axis='x', rotation=45)

# Deal size by outcome
deal_size_by_outcome = closed_deals.groupby('stage')['amount'].mean()
axes[1].bar(deal_size_by_outcome.index, deal_size_by_outcome.values)
axes[1].set_title('Average Deal Size by Outcome', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Average Amount ($)')

# Days to close by region
days_to_close = closed_deals[closed_deals['stage'] == 'Closed Won'].groupby('region')['days_to_close'].mean().sort_values()
axes[2].barh(days_to_close.index, days_to_close.values)
axes[2].set_title('Avg Days to Close (Won Deals)', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Days')

plt.tight_layout()
plt.show()

print(f"🏆 Overall Win Rate: {win_rate:.1f}%")
print(f"💰 Average Won Deal: ${closed_deals[closed_deals['stage'] == 'Closed Won']['amount'].mean():,.2f}")
print(f"⏱️  Average Sales Cycle: {closed_deals['days_to_close'].mean():.0f} days")

In [ ]:
# Monthly trends
opportunities['month'] = opportunities['created_date'].dt.to_period('M')
monthly_trends = opportunities.groupby('month').agg({
    'opportunity_id': 'count',
    'amount': 'sum',
    'weighted_amount': 'sum'
}).reset_index()
monthly_trends['month'] = monthly_trends['month'].astype(str)

fig, ax = plt.subplots(figsize=(12, 5))

ax.plot(monthly_trends['month'], monthly_trends['amount'], marker='o', linewidth=2, label='Total Pipeline')
ax.plot(monthly_trends['month'], monthly_trends['weighted_amount'], marker='s', linewidth=2, label='Weighted Pipeline')
ax.set_title('Monthly Pipeline Trends', fontsize=14, fontweight='bold')
ax.set_xlabel('Month')
ax.set_ylabel('Amount ($)')
ax.legend()
ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 💬 Part 2: Customer Feedback Analysis

Let's analyze customer satisfaction, response times, and feedback patterns.

In [ ]:
# Convert dates
feedback['submitted_date'] = pd.to_datetime(feedback['submitted_date'])

print("📋 Sample Feedback:")
feedback.head()

In [ ]:
# Rating distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Rating distribution
rating_counts = feedback['rating'].value_counts().sort_index()
axes[0].bar(rating_counts.index, rating_counts.values, color='skyblue', edgecolor='black')
axes[0].set_title('Customer Ratings Distribution', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Rating')
axes[0].set_ylabel('Count')
axes[0].set_xticks(range(1, 6))

# Average rating by feedback type
avg_rating_by_type = feedback.groupby('feedback_type')['rating'].mean().sort_values(ascending=False)
axes[1].bar(avg_rating_by_type.index, avg_rating_by_type.values, color='lightgreen', edgecolor='black')
axes[1].set_title('Average Rating by Feedback Type', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Average Rating')
axes[1].set_ylim(0, 5)

# Sentiment distribution
sentiment_counts = feedback['sentiment'].value_counts()
axes[2].pie(sentiment_counts.values, labels=sentiment_counts.index, autopct='%1.1f%%', startangle=90)
axes[2].set_title('Customer Sentiment', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"⭐ Average Rating: {feedback['rating'].mean():.2f}/5.0")
print(f"📝 Feedback with Comments: {feedback['has_comment'].sum():,} ({feedback['has_comment'].mean()*100:.1f}%)")

In [ ]:
# Response time analysis
responded = feedback[feedback['responded'] == True]

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Response time distribution
axes[0].hist(responded['response_days'].dropna(), bins=10, edgecolor='black', alpha=0.7)
axes[0].set_title('Response Time Distribution', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Days to Respond')
axes[0].set_ylabel('Count')

# Response rate by feedback type
response_rate = feedback.groupby('feedback_type')['responded'].mean() * 100
axes[1].bar(response_rate.index, response_rate.values, color='orange', edgecolor='black')
axes[1].set_title('Response Rate by Feedback Type', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Response Rate (%)')
axes[1].set_ylim(0, 100)

plt.tight_layout()
plt.show()

print(f"✅ Overall Response Rate: {feedback['responded'].mean()*100:.1f}%")
print(f"⏱️  Average Response Time: {responded['response_days'].mean():.1f} days")
print(f"⚡ Responded Within 2 Days: {feedback['responded_within_2days'].sum():,} ({feedback['responded_within_2days'].mean()*100:.1f}%)")

## 📦 Part 3: Inventory Analysis

Let's analyze inventory levels, value, and reorder needs.

In [ ]:
# Convert dates
inventory['last_updated'] = pd.to_datetime(inventory['last_updated'])

print("📋 Sample Inventory:")
inventory.head()

In [ ]:
# Inventory status and value
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Status distribution
status_counts = inventory['status'].value_counts()
axes[0,0].pie(status_counts.values, labels=status_counts.index, autopct='%1.1f%%', startangle=90)
axes[0,0].set_title('Inventory by Status', fontsize=12, fontweight='bold')

# Inventory value by category
value_by_category = inventory.groupby('category')['inventory_value'].sum().sort_values(ascending=False)
axes[0,1].bar(value_by_category.index, value_by_category.values, color='green', edgecolor='black')
axes[0,1].set_title('Inventory Value by Category', fontsize=12, fontweight='bold')
axes[0,1].set_ylabel('Total Value ($)')
axes[0,1].tick_params(axis='x', rotation=45)

# Items needing reorder
reorder_needed = inventory['needs_reorder'].value_counts()
axes[1,0].pie(reorder_needed.values, labels=['OK', 'Needs Reorder'] if False in reorder_needed.index else reorder_needed.index, autopct='%1.1f%%', startangle=90)
axes[1,0].set_title('Items Needing Reorder', fontsize=12, fontweight='bold')

# Margin analysis
margin_by_category = inventory.groupby('category')['margin_percent'].mean().sort_values(ascending=False)
axes[1,1].bar(margin_by_category.index, margin_by_category.values, color='purple', edgecolor='black')
axes[1,1].set_title('Average Margin % by Category', fontsize=12, fontweight='bold')
axes[1,1].set_ylabel('Margin %')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Top products by value
top_products = inventory.nlargest(10, 'inventory_value')[['product', 'inventory_value', 'quantity', 'status']]

plt.figure(figsize=(10, 6))
plt.barh(top_products['product'], top_products['inventory_value'], color='teal')
plt.xlabel('Inventory Value ($)')
plt.title('Top 10 Products by Inventory Value', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("🏆 Top 5 Products by Value:")
for idx, row in top_products.head().iterrows():
    print(f"  • {row['product']}: ${row['inventory_value']:,.2f} ({row['quantity']} units)")

## 📈 Part 4: Cross-Domain Insights

Let's connect insights across sales, feedback, and inventory.

In [ ]:
# Product performance correlation
product_sales = opportunities[opportunities['stage'] == 'Closed Won'].groupby('product').agg({
    'amount': ['sum', 'count']
}).round(2)
product_sales.columns = ['total_revenue', 'units_sold']
product_sales = product_sales.reset_index()

# Merge with inventory
product_inventory = inventory.groupby('product').agg({
    'inventory_value': 'sum',
    'quantity': 'sum',
    'margin_percent': 'mean'
}).reset_index()

product_performance = pd.merge(product_sales, product_inventory, on='product', how='outer').fillna(0)

print("📊 Product Performance Dashboard:")
product_performance

In [ ]:
# Correlation heatmap
# Create metrics by product
product_metrics = product_performance.set_index('product')

plt.figure(figsize=(8, 6))
sns.heatmap(product_metrics.corr(), annot=True, cmap='coolwarm', center=0, square=True)
plt.title('Product Metrics Correlation', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 📊 Part 5: Executive Dashboard

Creating a consolidated view of key metrics for business leaders.

In [ ]:
# Create executive dashboard
fig, axes = plt.subplots(2, 3, figsize=(16, 8))
fig.suptitle('PowerApps Business Intelligence Dashboard', fontsize=16, fontweight='bold')

# 1. Sales pipeline
stage_counts = opportunities['stage'].value_counts()
axes[0,0].pie(stage_counts.values, labels=stage_counts.index, autopct='%1.0f%%', startangle=90)
axes[0,0].set_title('Sales Pipeline Distribution')

# 2. Customer satisfaction
sentiment_counts = feedback['sentiment'].value_counts()
axes[0,1].pie(sentiment_counts.values, labels=sentiment_counts.index, autopct='%1.0f%%', startangle=90)
axes[0,1].set_title('Customer Sentiment')

# 3. Inventory health
status_counts = inventory['status'].value_counts()
axes[0,2].pie(status_counts.values, labels=status_counts.index, autopct='%1.0f%%', startangle=90)
axes[0,2].set_title('Inventory Health')

# 4. Monthly sales trend
monthly_sales = opportunities[opportunities['stage'] == 'Closed Won'].groupby('month')['amount'].sum()
if len(monthly_sales) > 0:
    axes[1,0].plot(monthly_sales.index.astype(str), monthly_sales.values, marker='o')
    axes[1,0].set_title('Monthly Sales Revenue')
    axes[1,0].tick_params(axis='x', rotation=45)

# 5. Top regions
top_regions = opportunities.groupby('region')['amount'].sum().nlargest(3)
axes[1,1].barh(top_regions.index, top_regions.values)
axes[1,1].set_title('Top Regions by Revenue')

# 6. Reorder alert
reorder_count = inventory['needs_reorder'].sum()
total_items = len(inventory)
axes[1,2].bar(['OK', 'Needs Reorder'], [total_items - reorder_count, reorder_count], color=['green', 'red'])
axes[1,2].set_title('Reorder Status')
axes[1,2].set_ylabel('Number of Products')

plt.tight_layout()
plt.show()

In [ ]:
# Summary metrics table
summary = pd.DataFrame({
    'Metric': [
        'Total Pipeline Value',
        'Weighted Pipeline',
        'Win Rate',
        'Average Customer Rating',
        'Customer Response Rate',
        'Total Inventory Value',
        'Items Needing Reorder',
        'Average Margin'
    ],
    'Value': [
        f"${opportunities['amount'].sum():,.0f}",
        f"${opportunities['weighted_amount'].sum():,.0f}",
        f"{win_rate:.1f}%",
        f"{feedback['rating'].mean():.2f}/5.0",
        f"{feedback['responded'].mean()*100:.1f}%",
        f"${inventory['inventory_value'].sum():,.0f}",
        f"{inventory['needs_reorder'].sum():,}",
        f"{inventory['margin_percent'].mean():.1f}%"
    ]
})

print("📋 EXECUTIVE SUMMARY DASHBOARD")
print("=" * 40)
for idx, row in summary.iterrows():
    print(f"{row['Metric']:30} {row['Value']}")

## ✅ Key Insights

Based on our analysis, here are the key business insights:

### Sales Insights
- **Win Rate**: The overall win rate shows sales team effectiveness
- **Regional Performance**: Some regions outperform others in both volume and speed
- **Deal Size Impact**: Larger deals take longer to close but have higher win rates

### Customer Feedback Insights
- **Response Time Matters**: Faster responses correlate with higher ratings
- **Feedback Channels**: Different feedback types have different response patterns
- **Sentiment Trends**: Overall customer sentiment is positive

### Inventory Insights
- **Value Concentration**: Top products drive most inventory value
- **Reorder Alerts**: Several items need immediate attention
- **Margin Analysis**: Certain categories have significantly better margins

## 🔄 Next Steps

1. **Automate this notebook** to run daily with new data
2. **Export key charts** to a PowerPoint deck for stakeholders
3. **Set up alerts** for critical thresholds (low inventory, poor feedback)
4. **Build predictive models** to forecast sales and inventory needs

In [ ]:
# Save key visualizations (optional)
# plt.savefig('dashboard_summary.png', dpi=300, bbox_inches='tight')
# print("✅ Dashboard saved as 'dashboard_summary.png'")